This code is just to clean and process the data. We will be using pandas to clean and process it.
Later we will be setting up an automation for this process, once we have the merged dataset we can
just go ahead and run this.

In [2]:
#Importing lib's
import pandas as pd
from soupsieve.util import lower

In [3]:
#Input and Output paths
input_file = r"S:\Sem_4\Scripts\merged_limnology_data.xlsx"
output_file = r"S:\Sem_4\Output\cleaned_limnology_data.xlsx"

In [4]:
#Loading the merged dataset
df = pd.read_excel(input_file, sheet_name='Limnology')

In [5]:
#Displaying basic information
print("Data:")
print(df.head())
print(df.info())
print(df.describe())
print("Total number of null values:")
print(df.isnull().sum())

Data:
  Start_Time Finish_Time                 Date Organization Crew_Initials  \
0   09:05:00    10:20:00  2024-07-23 00:00:00          BLA      SP/RP/JS   
1   10:35:00    11:30:00  2024-07-23 00:00:00          BLA      SP/RP/JS   
2   09:30:00    10:30:00  2024-09-03 00:00:00          BLA         JS/SP   
3   11:05:00    12:00:00  2024-09-03 00:00:00          BLA         JS/SP   
4   09:45:00    10:30:00  2025-03-14 00:00:00          BLA         JS/RB   

  Lake_Name   Site_Name   Latitude  Longitude Depth (in metres)  ...  \
0  Bob_Lake  BOBL-WQ-01  44.915948 -78.787557              62.1  ...   
1  Bob_Lake  BOBL-WQ-02  44.907262 -78.774812              30.7  ...   
2  Bob_Lake  BOBL-WQ-01   44.91466  -78.78398              61.8  ...   
3  Bob_Lake  BOBL-WQ-02   44.90886  -78.77561              30.2  ...   
4  Bob_Lake  BOBL-WQ-02   44.90886  -78.77561              30.1  ...   

  Collection_at_depth Depth Total_Kjeldahl_Nitrogen (mg/L) Ammonia (mg/L)  \
0                 yes   5.1

In [6]:
#Let's standardize column names to lower case and add "_" instead of " ".
df.columns = [col.lower().replace(" ", "_") for col in df.columns]

In [7]:
#Let's look for duplicate values, we probably will not have any duplicates.
duplicate_count = df.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)
#print(df.columns.tolist())

Number of duplicate rows: 0


In [8]:
#Now lets drop the columns which we won't be required for analysis.
df = df.drop(columns=['start_time', 'finish_time', 'organization', 'crew_initials', 'ph_strips', 'determination_1', 'determination_2', 'collection_at_depth'])

In [9]:
#We need to change the data type for most of the column as they are in-correct

#Lets convert the "date" column into datetype64 format and handel missing values
df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['date'] = df['date'].fillna('')

#Now lets convert few columns into string
string_columns = ['lake_name', 'site_name', 'air_moisture', 'cloud_cover', 'weather_last_24h', 'water_surface', 'color_disc']
for col in string_columns:
    if col in df.columns:
        df[col] = df[col].fillna('').astype(str)

#Cleaning "lake_name"
df['lake_name'] = (df['lake_name'].str.lower().str.replace('_', ' ')
                   .apply(lambda x: x if x.endswith(' lake') else x + ' lake')
)

#Now lets clean these columns like adding ',' instead of '/,-,_'
for col in ['air_moisture', 'cloud_cover', 'weather_last_24h', 'water_surface', 'color_disc']:
    if col in df.columns:
        df[col] = df[col].str.lower()
        df[col] = df[col].fillna('').astype(str)
        df[col] = df[col].str.replace('/', ',')
        df[col] = df[col].str.replace('-', ',')
        df[col] = df[col].str.replace('_', ',')
        #This code is to remove any cells which has ','
        df[col] = df[col].apply(lambda x: '' if x.strip(', ') == '' else x.strip(', '))

#For site name, we want to replace '_' by '-'
for col in ['site_name']:
    if col in df.columns:
        df[col] = df[col].str.replace('_', '-')

#For Stocking Lake, there a cell under weather_last_24h, I will be removing that cell as it is not consistent and does not fit.
df.loc[320, 'weather_last_24h'] = ''

#When we run df.info(), it will still show as object data type. We run the above code just in case if there is any discrepancy.

#Before converting the rest of the columns, we need to clean few column as they have '<' in the data.
#Function to remove "<"
def clean_numeric(value):
    if isinstance(value, str) and value.startswith('<'):
        try:
            return float(value[1:])
        except ValueError:
            return float('nan')
    return value

for col in ['ammonia_(mg/l)', 'ammonia_(mg/l)',  'nitrate_(mg/l)', 'nitrite_(mg/l)',
            'total_phosphorus_(mg/l)', 'total_phosphorus_duplicate',
            'total_phosphorus_(mg/l).1', 'sulfate_(mg/l)']:
    if col in df.columns:
        df[col] = df[col].apply(clean_numeric)
        df[col] = pd.to_numeric(df[col], errors='coerce')

#For columns phosphorus, there were few cases where the second reading of phosphorus is missing(they were not taken at the time of water quality checks). There were few cases were the first reading is missing and only average reading is present or the first one is present and other two are missing. So, if we have an average reading then we are keeping it, if we don't have average reading we will be looking at 1st and 2nd reading and taking their average and if anyone of them is missing we will use the present reading as the average reading.
phosphorus1 = 'total_phosphorus_(mg/l)'
phosphorus2 = 'total_phosphorus_duplicate'
average_phosphorus = 'total_phosphorus_(mg/l).1'

#Function to fill the average_phosphorus column
def fill_average(row):
    if pd.notnull(row[average_phosphorus]):
        return row[average_phosphorus]
    elif pd.notnull(row[phosphorus1]) and pd.notnull(row[phosphorus2]):
        return (row[phosphorus1] + row[phosphorus2]) / 2
    elif pd.notnull(row[phosphorus1]):
        return row[phosphorus1]
    elif pd.notnull(row[phosphorus2]):
        return row[phosphorus2]
    return float('nan')

if all(col in df.columns for col in ['total_phosphorus_(mg/l)', 'total_phosphorus_duplicate',
                                     'total_phosphorus_(mg/l).1']):
    df[average_phosphorus] = df.apply(fill_average, axis=1)
    df = df.drop(columns=[phosphorus1, phosphorus2], errors='ignore')
else:
    print("One or more phosphorus columns missing")

#Renaming the column for easier understanding
df.rename(columns={'total_phosphorus_(mg/l).1': 'average_phosphorus_(mg/l)'}, inplace=True)
df.rename(columns={'depth_(in_metres)': 'lake_depth_(in_metres)'}, inplace=True)


#Now lets convert the rest of the columns into float
lst_convert_flt = ['latitude', 'longitude', 'lake_depth_(in_metres)', 'air_temp', 'conductivity_(us/cm)',
                   'ph_meter', 'secchi_depth', 'depth', 'alkalinity', 'total_kjeldahl_nitrogen_(mg/l)']

for col in lst_convert_flt:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')


In [10]:
# Print column info after conversion
print("\nAfter all conversions:")
print(df.info())


After all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   date                            328 non-null    object 
 1   lake_name                       328 non-null    object 
 2   site_name                       328 non-null    object 
 3   latitude                        307 non-null    float64
 4   longitude                       310 non-null    float64
 5   lake_depth_(in_metres)          325 non-null    float64
 6   air_temp                        305 non-null    float64
 7   air_moisture                    328 non-null    object 
 8   cloud_cover                     328 non-null    object 
 9   weather_last_24h                328 non-null    object 
 10  water_surface                   328 non-null    object 
 11  conductivity_(us/cm)            326 non-null    float64
 12  chlorine    

In [11]:
df.head(10)

,date,lake_name,site_name,latitude,longitude,lake_depth_(in_metres),air_temp,air_moisture,cloud_cover,weather_last_24h,...,alkalinity,secchi_depth,color_disc,depth,total_kjeldahl_nitrogen_(mg/l),ammonia_(mg/l),nitrate_(mg/l),nitrite_(mg/l),average_phosphorus_(mg/l),sulfate_(mg/l)
0,2024-07-23,bob lake,BOBL-WQ-01,44.915948,-78.787557,62.1,22.6,dry,clear,clear,...,40.0,5.100,"yellow,brown",5.10,0.270,0.0050,0.02,NaN,0.002600,2.28
1,2024-07-23,bob lake,BOBL-WQ-02,44.907262,-78.774812,30.7,23.4,dry,clear,clear,...,70.0,5.100,"yellow,brown",5.10,0.237,0.0050,0.02,NaN,0.002200,2.26
2,2024-09-03,bob lake,BOBL-WQ-01,44.914660,-78.783980,61.8,18.4,dry,clear,clear,...,80.0,4.175,"yellow,brown",4.20,0.338,0.0271,0.02,NaN,0.005600,2.37
3,2024-09-03,bob lake,BOBL-WQ-02,44.908860,-78.775610,30.2,18.1,dry,clear,clear,...,60.0,4.100,"yellow,brown",4.10,0.263,0.0117,0.02,NaN,0.003900,2.20
4,2025-03-14,bob lake,BOBL-WQ-02,44.908860,-78.775610,30.1,-2.0,dry,clear,clear,...,40.0,NaN,,4.00,0.299,0.0277,0.12,NaN,0.007595,2.56
5,2025-06-19,bob lake,BOBL-WQ-01,44.914660,-78.783980,62.8,20.0,"damp,drizzle",overcast,light rain,...,20.0,4.800,"yellow,brown",4.80,0.268,0.0129,0.02,NaN,0.005800,2.27
6,2025-06-19,bob lake,BOBL-WQ-02,44.908860,-78.775610,30.6,22.0,"drizzle,shower",overcast,rain,...,30.0,4.750,"yellow,brown",4.75,0.221,0.0065,0.02,NaN,0.004600,2.45
7,2024-06-17,boshkung lake,BOSH-WQ-01,45.066688,-78.731267,39.7,19.4,damp,partial,rain,...,60.0,5.800,"blue,green",5.80,0.221,0.0059,0.02,NaN,0.004700,3.47
8,2024-06-17,boshkung lake,BOSH-WQ-02,45.055077,-78.725694,66.2,24.7,damp,partial,rain,...,40.0,5.500,"blue,green",5.50,0.226,0.0089,0.02,NaN,0.003800,3.27
9,2024-09-16,boshkung lake,BOSH-WQ-01,45.067250,-78.730830,54.0,20.5,dry,clear,sunny,...,36.0,5.450,"yellow,brown",5.45,0.247,0.0095,0.02,NaN,0.004500,3.23


In [12]:
#Final check for missing values
print("\nMissing Values after Cleaning:")
print(df.isnull().sum())


Missing Values after Cleaning:
date                                0
lake_name                           0
site_name                           0
latitude                           21
longitude                          18
lake_depth_(in_metres)              3
air_temp                           23
air_moisture                        0
cloud_cover                         0
weather_last_24h                    0
water_surface                       0
conductivity_(us/cm)                2
chlorine                           16
ph_meter                            2
total_hardness                      1
alkalinity                          5
secchi_depth                       28
color_disc                          0
depth                              72
total_kjeldahl_nitrogen_(mg/l)      7
ammonia_(mg/l)                      4
nitrate_(mg/l)                      5
nitrite_(mg/l)                    187
average_phosphorus_(mg/l)           4
sulfate_(mg/l)                      4
dtype: int64


In [13]:
#Export the file
df.to_excel(output_file, index=False, sheet_name='Limnology')